In [17]:
import tkinter as tk
from tkinter import ttk, messagebox
import requests
from PIL import Image, ImageTk
import os

In [18]:
API_URL = "http://127.0.0.1:8000"

In [19]:
COLOR_BG = "#FFFFFF"
COLOR_BG2 = "#D2DFFF"
COLOR_ACCENT = "#355CBD"
FONT_TITLE = ("Candara", 20, "bold")
FONT_HEADER = ("Candara", 14, "bold")
FONT_MAIN = ("Candara", 12)

In [ ]:
# GUI
def safe_get_json(url, params=None, timeout=8):
    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        return r.json()
    except requests.RequestException as e:
        raise RuntimeError(str(e))

def safe_post(url, json_data):
    try:
        r = requests.post(url, json=json_data, timeout=8)
        r.raise_for_status()
        return r
    except requests.RequestException as e:
        raise RuntimeError(str(e))

def safe_put(url, json_data):
    try:
        r = requests.put(url, json=json_data, timeout=8)
        r.raise_for_status()
        return r
    except requests.RequestException as e:
        raise RuntimeError(str(e))

def safe_delete(url):
    try:
        r = requests.delete(url, timeout=8)
        r.raise_for_status()
        return r
    except requests.RequestException as e:
        raise RuntimeError(str(e))


class App:
    def __init__(self, root):
        self.root = root
        root.title("Система продукции — Компания «Комфорт»")
        root.geometry("1200x760")
        root.configure(bg=COLOR_BG)

       
        try:
            if os.path.exists("logo.ico"):
                root.iconbitmap("logo.ico")
        except Exception:
            pass

        
        self.top = tk.Frame(root, bg=COLOR_BG2, height=120)
        self.top.pack(fill="x")
        self._load_logo_and_title()

        
        self.main = tk.Frame(root, bg=COLOR_BG)
        self.main.pack(fill="both", expand=True)

        self.left = tk.Frame(self.main, bg=COLOR_BG2, width=260)
        self.left.pack(side="left", fill="y")

        self.content = tk.Frame(self.main, bg=COLOR_BG)
        self.content.pack(side="right", fill="both", expand=True)

       
        self.footer = tk.Frame(root, bg=COLOR_BG2, height=28)
        self.footer.pack(fill="x")
        tk.Label(self.footer, text="© 2006–2025 Компания «Комфорт»", bg=COLOR_BG2, fg=COLOR_ACCENT, font=("Candara",10)).pack(side="right", padx=12)

        
        self._create_menu()

        
        self.screen_products()

    def _load_logo_and_title(self):
        try:
            if os.path.exists("logo.png"):
                img = Image.open("logo.png")
                w, h = img.size
                scale = 90.0 / h if h != 0 else 1.0
                img = img.resize((int(w * scale), 90), Image.LANCZOS)
                self.logo_img = ImageTk.PhotoImage(img)
                tk.Label(self.top, image=self.logo_img, bg=COLOR_BG2).pack(side="left", padx=12, pady=10)
            else:
                raise FileNotFoundError
        except Exception:
            tk.Label(self.top, text="Компания «Комфорт»", font=FONT_TITLE, bg=COLOR_BG2, fg=COLOR_ACCENT).pack(side="left", padx=12, pady=20)
        tk.Label(self.top, text="Управление продукцией", bg=COLOR_BG2, fg=COLOR_ACCENT, font=FONT_HEADER).pack(side="left", padx=8)

    def _create_menu(self):
        buttons = [
            ("Продукция", self.screen_products),
            ("Типы продукции", self.screen_product_types),
            ("Материалы", self.screen_materials),
            ("Цеха", self.screen_workshops),
            ("Процесс", self.screen_process),
            ("Время изготовления", self.screen_time),
            ("Расчёт сырья", self.screen_raw),
        ]
        for txt, cmd in buttons:
            btn = tk.Button(self.left, text=txt, command=cmd, bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=24)
            btn.pack(pady=6, padx=10)

    def clear(self):
        for w in self.content.winfo_children():
            w.destroy()

   
    def screen_products(self):
        self.clear()
        tk.Label(self.content, text="Список продукции", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)

        try:
            data = safe_get_json(f"{API_URL}/products")
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось получить список продукции: {e}")
            return

        if not data:
            tk.Label(self.content, text="Нет данных", bg=COLOR_BG).pack()
            return

        cols = list(data[0].keys())
        self.tree_products = ttk.Treeview(self.content, columns=cols, show="headings", height=18)
        self.tree_products.pack(fill="both", expand=True, padx=16, pady=8)

        vsb = ttk.Scrollbar(self.content, orient="vertical", command=self.tree_products.yview)
        self.tree_products.configure(yscrollcommand=vsb.set)
        vsb.place(in_=self.tree_products, relx=1.0, rely=0, relheight=1.0, anchor="ne")

        for c in cols:
            self.tree_products.heading(c, text=c)
            self.tree_products.column(c, anchor="center", width=140)

        for row in data:
            self.tree_products.insert("", "end", values=[row.get(c) for c in cols])

        panel = tk.Frame(self.content, bg=COLOR_BG)
        panel.pack(pady=8)
        tk.Button(panel, text="Добавить", command=lambda: self.open_product_editor(None), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=12).pack(side="left", padx=6)
        tk.Button(panel, text="Редактировать", command=self._edit_selected_product, bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=12).pack(side="left", padx=6)
        tk.Button(panel, text="Удалить", command=self._delete_selected_product, bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, width=12).pack(side="left", padx=6)

    def _get_selected_from_tree(self, tree):
        sel = tree.selection()
        if not sel:
            return None
        vals = tree.item(sel[0])["values"]
        cols = tree["columns"]
        return {cols[i]: vals[i] for i in range(len(cols))}

    def _edit_selected_product(self):
        obj = self._get_selected_from_tree(self.tree_products)
        if not obj:
            messagebox.showwarning("Выбор", "Выберите запись")
            return
        self.open_product_editor(obj)

    def _delete_selected_product(self):
        obj = self._get_selected_from_tree(self.tree_products)
        if not obj:
            messagebox.showwarning("Выбор", "Выберите запись")
            return
        pid = obj.get("product_id")
        name = obj.get("product_name")
        if messagebox.askyesno("Удалить", f"Удалить {name} (ID {pid})?"):
            try:
                safe_delete(f"{API_URL}/products/{pid}")
                messagebox.showinfo("OK", "Удалено")
                self.screen_products()
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))

    def open_product_editor(self, product):
        win = tk.Toplevel(self.root)
        win.title("Добавить / Редактировать продукцию")
        win.geometry("520x520")
        win.configure(bg=COLOR_BG)

        tk.Label(win, text="Продукция", font=FONT_HEADER, bg=COLOR_BG).pack(pady=8)
        tk.Label(win, text="Название", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_name = tk.Entry(win, font=FONT_MAIN, width=60); e_name.pack(padx=12, pady=4)
        tk.Label(win, text="Артикул", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_article = tk.Entry(win, font=FONT_MAIN, width=40); e_article.pack(padx=12, pady=4)
        tk.Label(win, text="Минимальная цена", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_price = tk.Entry(win, font=FONT_MAIN, width=20); e_price.pack(padx=12, pady=4)

        try:
            types = safe_get_json(f"{API_URL}/product-types")
        except Exception:
            types = []
        try:
            mats = safe_get_json(f"{API_URL}/material-types")
        except Exception:
            mats = []

        tk.Label(win, text="Тип продукции", bg=COLOR_BG).pack(anchor="w", padx=12)
        cb_type = ttk.Combobox(win, state="readonly", values=[f"{t['product_type_id']} - {t['type_name']}" for t in types], font=FONT_MAIN, width=40)
        cb_type.pack(padx=12, pady=4)

        tk.Label(win, text="Материал", bg=COLOR_BG).pack(anchor="w", padx=12)
        cb_mat = ttk.Combobox(win, state="readonly", values=[f"{m['material_type_id']} - {m['material_name']}" for m in mats], font=FONT_MAIN, width=40)
        cb_mat.pack(padx=12, pady=4)

        edit_id = None
        if product:
            edit_id = product.get("product_id")
            e_name.insert(0, product.get("product_name") or "")
            e_article.insert(0, product.get("article_number") or "")
            e_price.insert(0, str(product.get("min_partner_price") or ""))
            ptype = product.get("product_type")
            for t in types:
                if t.get("type_name") == ptype:
                    cb_type.set(f"{t['product_type_id']} - {t['type_name']}")
            mname = product.get("material_name")
            for m in mats:
                if m.get("material_name") == mname:
                    cb_mat.set(f"{m['material_type_id']} - {m['material_name']}")

        def save():
            name = e_name.get().strip(); art = e_article.get().strip(); price_s = e_price.get().strip()
            if not name or not art or not price_s:
                messagebox.showerror("Ошибка", "Заполните обязательные поля"); return
            try:
                price = float(price_s)
                if price <= 0: raise ValueError
            except:
                messagebox.showerror("Ошибка", "Неверная цена"); return
            if not cb_type.get() or not cb_mat.get():
                messagebox.showerror("Ошибка", "Выберите тип и материал"); return
            type_id = int(cb_type.get().split(" - ")[0])
            mat_id = int(cb_mat.get().split(" - ")[0])
            payload = {"product_type_id": type_id, "product_name": name, "article_number": art, "min_partner_price": price, "material_type_id": mat_id}
            try:
                if edit_id:
                    safe_put(f"{API_URL}/products/{edit_id}", payload)
                else:
                    safe_post(f"{API_URL}/products", payload)
                messagebox.showinfo("OK", "Сохранено")
                win.destroy()
                self.screen_products()
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))

        btns = tk.Frame(win, bg=COLOR_BG); btns.pack(pady=10)
        tk.Button(btns, text="Сохранить", bg=COLOR_ACCENT, fg="white", command=save, font=FONT_MAIN).pack(side="left", padx=6)
        tk.Button(btns, text="Отмена", bg="#777777", fg="white", command=win.destroy, font=FONT_MAIN).pack(side="left", padx=6)

    
    def screen_product_types(self):
        self.clear()
        tk.Label(self.content, text="Типы продукции", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            data = safe_get_json(f"{API_URL}/product-types")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return
        cols = list(data[0].keys()) if data else ["product_type_id", "type_name", "coefficient"]
        tree = ttk.Treeview(self.content, columns=cols, show="headings", height=18)
        tree.pack(fill="both", expand=True, padx=16, pady=8)
        for c in cols:
            tree.heading(c, text=c); tree.column(c, anchor="center", width=180)
        for row in data:
            tree.insert("", "end", values=[row.get(k) for k in cols])
        panel = tk.Frame(self.content, bg=COLOR_BG); panel.pack(pady=8)
        tk.Button(panel, text="Добавить", command=lambda: self._open_type_editor(None), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN).pack(side="left", padx=6)
        tk.Button(panel, text="Редактировать", command=lambda: self._edit_type_selected(tree), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN).pack(side="left", padx=6)
        tk.Button(panel, text="Удалить", command=lambda: self._delete_type_selected(tree), bg=COLOR_ACCENT, fg="white", font=FONT_MAIN).pack(side="left", padx=6)

    def _open_type_editor(self, obj):
        win = tk.Toplevel(self.root); win.title("Тип продукции"); win.geometry("420x260"); win.configure(bg=COLOR_BG)
        tk.Label(win, text="Тип продукции", font=FONT_HEADER, bg=COLOR_BG).pack(pady=8)
        tk.Label(win, text="Название", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_name = tk.Entry(win, font=FONT_MAIN, width=40); e_name.pack(padx=12, pady=4)
        tk.Label(win, text="Коэффициент", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_coef = tk.Entry(win, font=FONT_MAIN, width=20); e_coef.pack(padx=12, pady=4)
        edit_id = None
        if obj:
            edit_id = obj.get("product_type_id"); e_name.insert(0, obj.get("type_name") or ""); e_coef.insert(0, str(obj.get("coefficient") or ""))
        def save():
            name = e_name.get().strip(); coef_s = e_coef.get().strip()
            if not name or not coef_s:
                messagebox.showerror("Ошибка", "Заполните поля"); return
            try:
                coef = float(coef_s)
                if coef <= 0: raise ValueError
            except:
                messagebox.showerror("Ошибка", "Коэффициент должен быть положительным числом"); return
            payload = {"type_name": name, "coefficient": coef}
            try:
                if edit_id:
                    safe_put(f"{API_URL}/product-types/{edit_id}", payload)
                else:
                    safe_post(f"{API_URL}/product-types", payload)
                messagebox.showinfo("OK", "Сохранено"); win.destroy(); self.screen_product_types()
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))
        btns = tk.Frame(win, bg=COLOR_BG); btns.pack(pady=8)
        tk.Button(btns, text="Сохранить", bg=COLOR_ACCENT, fg="white", command=save, font=FONT_MAIN).pack(side="left", padx=6)
        tk.Button(btns, text="Отмена", bg="#777777", fg="white", command=win.destroy, font=FONT_MAIN).pack(side="left", padx=6)

    def _edit_type_selected(self, tree):
        sel = tree.selection()
        if not sel:
            messagebox.showwarning("Выбор", "Выберите строку"); return
        vals = tree.item(sel[0])["values"]; obj = {"product_type_id": vals[0], "type_name": vals[1], "coefficient": vals[2]}; self._open_type_editor(obj)

    def _delete_type_selected(self, tree):
        sel = tree.selection()
        if not sel:
            messagebox.showwarning("Выбор", "Выберите строку"); return
        vals = tree.item(sel[0])["values"]; tid, name = vals[0], vals[1]
        if not messagebox.askyesno("Удалить", f"Удалить тип {name} (ID {tid})?"): return
        try:
            safe_delete(f"{API_URL}/product-types/{tid}")
            messagebox.showinfo("OK", "Удалено"); self.screen_product_types()
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    
    def screen_materials(self):
        self.clear()
        tk.Label(self.content, text="Типы материалов", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            data = safe_get_json(f"{API_URL}/material-types")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return
        cols = list(data[0].keys()) if data else ["material_type_id", "material_name", "loss_percentage"]
        tree = ttk.Treeview(self.content, columns=cols, show="headings", height=18); tree.pack(fill="both", expand=True, padx=16, pady=8)
        for c in cols: tree.heading(c, text=c); tree.column(c, anchor="center", width=180)
        for row in data: tree.insert("", "end", values=[row.get(k) for k in cols])

    
    def screen_workshops(self):
        self.clear()
        tk.Label(self.content, text="Цеха", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            data = safe_get_json(f"{API_URL}/workshops")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return
        cols = list(data[0].keys()) if data else ["workshop_id", "workshop_name", "category_name", "staff_count"]
        tree = ttk.Treeview(self.content, columns=cols, show="headings", height=18); tree.pack(fill="both", expand=True, padx=16, pady=8)
        for c in cols: tree.heading(c, text=c); tree.column(c, anchor="center", width=160)
        for row in data: tree.insert("", "end", values=[row.get(k) for k in cols])

    
    def screen_process(self):
        self.clear()
        tk.Label(self.content, text="Производственный процесс", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)

       
        try:
            products = safe_get_json(f"{API_URL}/products")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return

        frame = tk.Frame(self.content, bg=COLOR_BG)
        frame.pack(fill="x", padx=12, pady=4)

        tk.Label(frame, text="Продукция:", bg=COLOR_BG).pack(side="left", padx=6)
        cb_products = ttk.Combobox(frame, values=[f"{p['product_id']} - {p['product_name']}" for p in products], state="readonly", width=80, font=FONT_MAIN)
        cb_products.pack(side="left", padx=6)

        
        cols = ("process_id", "workshop_name", "category_name", "staff_count", "manufacturing_hours", "workshop_id")
        tree = ttk.Treeview(self.content, columns=cols, show="headings", height=18)
        for c in cols:
            tree.heading(c, text=c)
            
            width = 120 if c != "workshop_id" else 0
            tree.column(c, anchor="center", width=width)
        tree.pack(fill="both", expand=True, padx=12, pady=8)

        vs = ttk.Scrollbar(self.content, orient="vertical", command=tree.yview)
        tree.configure(yscrollcommand=vs.set)
        vs.place(in_=tree, relx=1.0, rely=0, relheight=1.0, anchor="ne")

       
        panel = tk.Frame(self.content, bg=COLOR_BG)
        panel.pack(pady=6)
        tk.Button(panel, text="Добавить этап", bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, command=lambda: self._add_stage(cb_products, tree, products)).pack(side="left", padx=6)
        tk.Button(panel, text="Удалить этап", bg=COLOR_ACCENT, fg="white", font=FONT_MAIN, command=lambda: self._delete_stage(tree)).pack(side="left", padx=6)

        def load_for_product(event=None):
            if not cb_products.get():
                return
            product_id = int(cb_products.get().split(" - ")[0])
            try:
                rows = safe_get_json(f"{API_URL}/production-process/{product_id}")
            except Exception as e:
                messagebox.showerror("Ошибка", str(e)); return
            tree.delete(*tree.get_children())
            for r in rows:
                tree.insert("", "end", values=[r.get("process_id"), r.get("workshop_name"), r.get("category_name"), r.get("staff_count"), r.get("manufacturing_hours"), r.get("workshop_id")])

        cb_products.bind("<<ComboboxSelected>>", load_for_product)

    def _add_stage(self, cb_products, tree, products_list):
        if not cb_products.get():
            messagebox.showwarning("Выбор", "Выберите продукцию")
            return
        product_id = int(cb_products.get().split(" - ")[0])

        dlg = tk.Toplevel(self.root)
        dlg.title("Добавить этап")
        dlg.geometry("420x220")
        dlg.configure(bg=COLOR_BG)

        tk.Label(dlg, text="Добавить этап для продукта", font=FONT_HEADER, bg=COLOR_BG).pack(pady=8)
        tk.Label(dlg, text="Цех:", bg=COLOR_BG).pack(anchor="w", padx=12)

        try:
            all_workshops = safe_get_json(f"{API_URL}/workshops")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); dlg.destroy(); return

        
        try:
            existing = safe_get_json(f"{API_URL}/workshops_for_product/{product_id}")
            existing_ids = {r['workshop_id'] for r in existing}
        except Exception:
            existing_ids = set()

        options = [w for w in all_workshops if w['workshop_id'] not in existing_ids]
        if not options:
            messagebox.showinfo("Инфо", "У этого продукта уже добавлены все цеха.")
            dlg.destroy()
            return

        cb_ws = ttk.Combobox(dlg, values=[f"{w['workshop_id']} - {w['workshop_name']}" for w in options], state="readonly", width=50)
        cb_ws.pack(padx=12, pady=6)

        tk.Label(dlg, text="Время (целые часы):", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_hours = tk.Entry(dlg, width=10); e_hours.pack(padx=12, pady=6)
        e_hours.insert(0, "1")

        def do_add():
            if not cb_ws.get() or not e_hours.get().strip():
                messagebox.showerror("Ошибка", "Заполните все поля"); return
            try:
                hours = int(float(e_hours.get().strip()))
                if hours <= 0:
                    raise ValueError
            except Exception:
                messagebox.showerror("Ошибка", "Время должно быть положительным целым"); return
            wid = int(cb_ws.get().split(" - ")[0])
            payload = {"workshop_id": wid, "manufacturing_hours": hours}
            try:
                safe_post(f"{API_URL}/production-process/{product_id}", payload)
                messagebox.showinfo("OK", "Этап добавлен")
                dlg.destroy()
              
                self.screen_process()  
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))

        tk.Button(dlg, text="Сохранить", bg=COLOR_ACCENT, fg="white", command=do_add, font=FONT_MAIN).pack(pady=8)
        tk.Button(dlg, text="Отмена", bg="#777777", fg="white", command=dlg.destroy, font=FONT_MAIN).pack()

    def _delete_stage(self, tree):
        sel = tree.selection()
        if not sel:
            messagebox.showwarning("Выбор", "Выберите этап для удаления"); return
        vals = tree.item(sel[0])["values"]
        process_id = vals[0]
        if not messagebox.askyesno("Удалить", f"Удалить этап ID={process_id}?"):
            return
        try:
            safe_delete(f"{API_URL}/production-process/{process_id}")
            messagebox.showinfo("OK", "Этап удалён")
            tree.delete(sel[0])
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    
    def screen_time(self):
        self.clear()
        tk.Label(self.content, text="Расчёт времени изготовления", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            prods = safe_get_json(f"{API_URL}/products")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return
        cb = ttk.Combobox(self.content, values=[f"{p['product_id']} - {p['product_name']}" for p in prods], font=FONT_MAIN, width=80)
        cb.pack(pady=6)
        def calc_time():
            if not cb.get(): messagebox.showwarning("Ошибка", "Выберите продукт"); return
            pid = int(cb.get().split(" - ")[0])
            try:
                r = safe_get_json(f"{API_URL}/manufacture-time/{pid}")
                hours = r.get("manufacture_time_hours", 0)
                messagebox.showinfo("Время изготовления", f"Продукт ID {pid}: {hours} часов")
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))
        tk.Button(self.content, text="Рассчитать", bg=COLOR_ACCENT, fg="white", command=calc_time).pack(pady=8)

    
    def screen_raw(self):
        self.clear()
        tk.Label(self.content, text="Расчёт сырья", font=FONT_TITLE, bg=COLOR_BG, fg=COLOR_ACCENT).pack(pady=10)
        try:
            ptypes = safe_get_json(f"{API_URL}/product-types")
            mtypes = safe_get_json(f"{API_URL}/material-types")
        except Exception as e:
            messagebox.showerror("Ошибка", str(e)); return

        tk.Label(self.content, text="Тип продукции", bg=COLOR_BG).pack(anchor="w", padx=12)
        cbp = ttk.Combobox(self.content, values=[f"{p['product_type_id']} - {p['type_name']}" for p in ptypes], font=FONT_MAIN, width=60)
        cbp.pack(padx=12, pady=4)

        tk.Label(self.content, text="Материал", bg=COLOR_BG).pack(anchor="w", padx=12)
        cbm = ttk.Combobox(self.content, values=[f"{m['material_type_id']} - {m['material_name']}" for m in mtypes], font=FONT_MAIN, width=60)
        cbm.pack(padx=12, pady=4)

        tk.Label(self.content, text="Количество (шт)", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_qty = tk.Entry(self.content, font=FONT_MAIN); e_qty.pack(padx=12, pady=4)

        tk.Label(self.content, text="Длина изделия", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_p1 = tk.Entry(self.content, font=FONT_MAIN); e_p1.pack(padx=12, pady=4)

        tk.Label(self.content, text="Ширина изделия", bg=COLOR_BG).pack(anchor="w", padx=12)
        e_p2 = tk.Entry(self.content, font=FONT_MAIN); e_p2.pack(padx=12, pady=4)

        def calc_raw():
            try:
                pid = int(cbp.get().split(" - ")[0])
                mid = int(cbm.get().split(" - ")[0])
                qty = int(e_qty.get())
                p1 = float(e_p1.get())
                p2 = float(e_p2.get())
            except Exception:
                messagebox.showerror("Ошибка", "Проверьте ввод"); return
            try:
                r = safe_get_json(f"{API_URL}/calc-raw", params={"product_type_id": pid, "material_type_id": mid, "quantity": qty, "p1": p1, "p2": p2})
                if r.get("result", -1) == -1:
                    messagebox.showerror("Ошибка", "Параметры недопустимы")
                else:
                    messagebox.showinfo("Результат", f"Необходимое количество сырья: {r.get('result')}")
            except Exception as e:
                messagebox.showerror("Ошибка", str(e))

        tk.Button(self.content, text="Рассчитать", bg=COLOR_ACCENT, fg="white", command=calc_raw).pack(pady=8)

def main():
    root = tk.Tk()
    app = App(root)
    root.mainloop()

if __name__ == "__main__":
    main()